In [1]:
import ee
import folium

In [5]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # Inherit from parent class.
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # Create vector feature.
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining.
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a Google Earth Engine raster layer to a folium map.
        """       
        # Handle ImageCollections and Images.
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # Get the JSON instructions to show image tiles.
        map_id_dict = image.getMapId(vis_params)
        
        # Get url of the raster tiles.
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # Create a folium raster layer and add to self.
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # Add raster to Map and return self to allow chaining.
        self.add_child(raster)
        return self

In [17]:
# Load Hansen dataset raster.
forest = ee.Image('UMD/hansen/global_forest_change_2015')

# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Add raster to map.
emap.add_ee_raster(forest)

emap

In [49]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Add grayscale layer to map.
emap.add_ee_raster(
    image = forest,
    name = "Grayscale",
    attr = "Grayscale",
    vis_params={
        "bands" : ['treecover2000'],
        "max" : 100,
        "palette": ['black', 'white'],
    },
)

emap

In [35]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Add false color layer from three bands.
emap.add_ee_raster(
    image = forest,
    name = "False color",
    attr = "False color",
    vis_params={
        "bands": ['last_b50', 'last_b40', 'last_b30'],
        "min": 0,
        "max": 255
    },
)

emap

In [36]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Show forest extent as green, loss as red and gain as blue.
emap.add_ee_raster(
    image = forest,
    name = "Gain and loss",
    attr = "Gain and loss",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
        "min": 0,
        "max": 100
    },
)

emap

In [43]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Set stretching ranges for tree cover, loss and gain.
emap.add_ee_raster(
    image = forest,
    name = "Gain and loss",
    attr = "Gain and loss",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
        "max": [1, 255, 1]
    },
)

emap

In [42]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Use palette to select color for forest cover, plus adjusting brightness.
emap.add_ee_raster(
    image = forest,
    name = "Forest cover palette",
    attr = "Forest cover palette",
    vis_params={
        "bands": ['treecover2000'],
        "max" : 100,
        "palette" : ['black', 'green']
    },
)

emap

In [51]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Use masking to make black areas transparent.
emap.add_ee_raster(
    image = forest.mask(forest),
    name = "Masked forest cover palette",
    attr = "Masked forest cover palette",
    vis_params={
        "bands": ['treecover2000'],
        "max" : 100,
        "palette" : ['black', 'green']
    },
)

emap

In [55]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Select bands.
treecover = forest.select('treecover2000')
loss = forest.select('loss')
gain = forest.select('gain')

# Add rasters to map with masking.
emap.add_ee_raster(
    image = treecover.updateMask(treecover),
    name = "Treecover",
    attr = "Treecover",
    vis_params={
        "bands": ['treecover2000'],
        "max" : 100,
        "palette" : ['black', 'green']
    },
)
emap.add_ee_raster(
    image = loss.updateMask(loss),
    name = "Loss",
    attr = "Loss",
    vis_params={
        "bands": ['loss'],
        "max" : 1,
        "palette" : ['red']
    },
)
emap.add_ee_raster(
    image = gain.updateMask(gain),
    name = "Gain",
    attr = "Gain",
    vis_params={
        "bands": ['gain'],
        "max" : 1,
        "palette" : ['blue']
    },
)

emap.add_child(folium.LayerControl())
emap

In [81]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Select bands.
gain = forest.select("gain")
loss = forest.select("loss")

# Combine into one band.
gain_and_loss = gain.add(loss)

# Add raster to map.
emap.add_ee_raster(
    image = gain_and_loss.updateMask(gain_and_loss),
    name = "Gain and loss",
    attr = "Gain and loss",
    vis_params = {
        "palette" : ["pink"]
    }
)

emap

In [82]:
# Create map.
emap = EngineMap(location=(40, -100), zoom_start = 4)

# Select bands.
treecover = forest.select('treecover2000')
loss = forest.select('loss')
gain = forest.select('gain')

# Make combined band.
gain_and_loss = gain.add(loss)

# Add rasters to map with masking.
emap.add_ee_raster(
    image = treecover.updateMask(treecover),
    name = "Treecover",
    attr = "Treecover",
    vis_params={
        "bands": ['treecover2000'],
        "max" : 100,
        "palette" : ['black', 'green']
    },
)
emap.add_ee_raster(
    image = loss.updateMask(loss),
    name = "Loss",
    attr = "Loss",
    vis_params={
        "bands": ['loss'],
        "max" : 1,
        "palette" : ['red']
    },
)
emap.add_ee_raster(
    image = gain.updateMask(gain),
    name = "Gain",
    attr = "Gain",
    vis_params={
        "bands": ['gain'],
        "max" : 1,
        "palette" : ['blue']
    },
)
emap.add_ee_raster(
    image = gain_and_loss.updateMask(gain_and_loss),
    name = "Gain and loss",
    attr = "Gain and loss (Henry)",
    vis_params = {
        "max" : 1,
        "palette" : ["pink"]
    }
)

emap.add_child(folium.LayerControl())
emap